In [ ]:
import pandas
import matplotlib.pyplot as plt
import seaborn as sns
import utils
import numpy as np

In [ ]:
###
# Effect of PEG on migration through gel.
###

df = pandas.read_excel('180522.xlsx', sheetname='PEG_1')

df['% PEG'] = [int(x*100)/100 for x in df['% PEG']]

norm = df.groupby(by=['% PEG', 'Channel']).mean()
g = norm.loc[(slice(None), 800),]['Signal'].tolist()
r = norm.loc[(slice(None), 700),]['Signal'].tolist()
ratio = []
peg_to_ratio = {}
for x,y,n in zip(g, r, norm.loc[(slice(None), 800),].index):
    peg_to_ratio[n[0]] = x/y

def normalize(signal, peg, channel):
    if channel == 800:
        return signal
    return 3 * signal

df['Signal'] = [normalize(x,y,z) for x,y,z in zip(df.Signal, df['% PEG'], df['Channel'])]

# Make figure.
plt.clf()
fig = plt.figure()
sns.set_style('ticks')
green = df[df['Channel']==800]
#print(df['Image Name'])
df = df[df['Image Name']=='0009890_01']
#sns.stripplot(data=df, y='Signal', x='% PEG')#, hue='Channel')
sns.barplot(data=df[df['Channel']==800], y='Signal', x='% PEG',
           palette=sns.cubehelix_palette(start=-.4, rot=-.14, n_colors=len(set(df['% PEG']))))
fig.set_figwidth(2.5)
fig.set_figheight(4)
utils.draw(fig, 'figs/peg_perc_vs_staple_signal_on_gel.pdf')

In [ ]:
###
# Effect of buffer on fluorescence after diluting.
###

# Figure S{effect_of_dilutions_and_carrier}
df = utils.load_sheet(
    fname='Shift based quantification protocol and results.xlsx',
    sheet_name='180429_1')

plt.clf()
fig, ax = plt.subplots()

#print(df.groupby(['Buffer', 'Object']).mean())
baseline = df.groupby(['Buffer', 'Object']).mean().loc['Water', 'αL3']['Signal/fmol']
#print(df.groupby(['Buffer', 'Object']).mean().loc['H2O', 'aL3']['Signal/fmol'])
#df['Signal/fmol'] = [x/baseline for x in df['Signal/fmol']]
#print(df.groupby(['Buffer', 'Object']).mean())
sns.barplot(
    data=df, x='FU/fmol', y='Object', hue='Buffer',
    palette=sns.cubehelix_palette(start=-.4, rot=-.14, n_colors=len(set(df['Buffer']))))
fig.savefig('figs/effect_of_dilutions_and_carrier.pdf')
plt.show()
plt.clf()

In [ ]:
###
# Effect of buffer on fluorescence after diluting - more detailed experiment.
###

obj_color = utils.obj_color

def stripbox(_df, filename, xcol='', ycol='', hue=None):
    ax = sns.stripplot(data=_df, x=xcol, y=ycol, hue=hue,
                      dodge=True, 
                      palette=obj_color
                      )
    ax = sns.boxplot(data=_df, x=xcol, y=ycol, hue=hue,
                    fliersize=0, boxprops=dict(alpha=.2, linewidth=0),
                     linewidth=0, palette=obj_color,
                     meanline=True, showmeans=True, #showmedians=False,
                    showfliers=False, showcaps=False, showbox=True, whis=0)

    fig.savefig(filename)
    plt.show()
    plt.clf()

    
df = utils.load_sheet(fname='Buffer blocking dot test.xlsx', sheet_name='180518_1')
df = df[df['fmols']>0.02]  # The lowest concentration is invalid due to backgound from wet spot.

by_tup = {}
for r in df.to_dict('records'):
    by_tup[(r['Salt'], r['fmols'], r['Blocking agent'], r['Replicate'])] = r

# Dilution effciency
for r in df.to_dict('records'):
    t = (r['Salt'], r['fmols'], r['Blocking agent'], r['Replicate'])
    prev = (r['Salt'], r['fmols']*10, r['Blocking agent'], r['Replicate'])
    if prev in by_tup:
        if by_tup[prev]['Signal'] <= 0:
            by_tup[t]['Retention'] = np.nan
        else:
            by_tup[t]['Retention'] = 10*by_tup[t]['Signal']/by_tup[prev]['Signal']
    else:
        by_tup[t]['Retention'] = 1

    by_tup[t]['Retention'] = max([0, by_tup[t]['Retention']])
    by_tup[t]['Retention'] = min([1, by_tup[t]['Retention']])

_l = []
for k, v in by_tup.items():
    _l.append(v)

df = pandas.DataFrame(_l)#.from_dict(by_tup, orient='index')

above = df[df['fmols']>0.2]

means_by_conc = above.groupby(by=['fmols', 'Salt', 'Blocking agent']).mean()

_ = df.groupby(by=['fmols', 'Salt', 'Blocking agent', 'Replicate']).mean()

brightness = []
for blocking in [
        'No blocking agent', '5 ng/ul', '10 ng/ul', '20 ng/ul',
        '10 ng/ul, 8% PEG400', '10 ng/ul, 16% PEG400', '10 ng/ul, 25% PEG400',
        '10 ng/ul, 8% PEG8000', '10 ng/ul, 16% PEG8000', '10 ng/ul, 25% PEG8000',
        ]:
    for salt in set(df['Salt']):

        dilution_efficiency = _.loc[(2, salt, blocking, slice(None)), 'Retention'].mean()

        a = _.loc[(20, salt, blocking,),]['Signal']/dilution_efficiency

        for rep in set(df['Replicate']):
            _d = {
                'Replicate': rep,
                'Brightness': _.loc[(20, salt, blocking, rep),]['Signal']/dilution_efficiency,
                'Dilution efficiency': dilution_efficiency,
                'Blocking agent': blocking,
                'Salt': salt,
            }
            brightness.append(_d)
            
b_df = pandas.DataFrame(brightness)

norm_bright = b_df.groupby(by=['Salt', 'Blocking agent']).mean()

norm_bright = norm_bright.loc[('50 mM NaCl', 'No blocking agent')]['Brightness']

b_df.Brightness = [x/norm_bright for x in b_df.Brightness]

stripbox(b_df, 'figs/Brightness_vs_blocking_agent_dots.pdf',
         xcol='Brightness', ycol='Blocking agent', hue='Salt')

# PEG only now.
with_peg = ['10 ng/ul, 8% PEG400', '10 ng/ul, 16% PEG400', '10 ng/ul, 25% PEG400',
        '10 ng/ul, 8% PEG8000', '10 ng/ul, 16% PEG8000', '10 ng/ul, 25% PEG8000',
        ]
peg = b_df[[(x in with_peg) for x in b_df['Blocking agent']]]

fig = plt.figure()
sns.barplot(data=peg, x='Brightness', y='Salt', palette=obj_color)
fig.set_figwidth(3)
fig.set_figheight(1)
utils.draw(fig, 'figs/brightness_vs_salt_dots_peg_only.pdf')

peg = df[[(x in with_peg) for x in df['Blocking agent']]]

fig = plt.figure()
sns.barplot(data=peg, x='Signal/fmol', y='Salt', palette=obj_color)
fig.set_figwidth(3)
fig.set_figheight(1)
utils.draw(fig, 'figs/signal_per_fmol_vs_salt_dots.pdf')

diluted_peg = peg[peg['fmols']<20]

fig = plt.figure()
sns.barplot(data=diluted_peg, x='Retention', y='Salt', palette=obj_color)
fig.set_figwidth(3)
fig.set_figheight(1)
utils.draw(fig, 'figs/retention_vs_salt_dots.pdf')



primary_conc = df[df['fmols']==20]

diluted = df[df['fmols']<20]
diluted = diluted[[(x in with_peg) for x in diluted['Blocking agent']]]

diluted_1_nM = diluted[diluted['fmols']==2]
diluted_100_pM = diluted[diluted['fmols']==0.2]
diluted_10_pM = diluted[diluted['fmols']==0.02]

plt.clf()

fig = plt.figure()
plt.title('10 nM')
stripbox(primary_conc, 'figs/10_nM_signal_per_fmol_vs_blocking_agent_dots.pdf',
         xcol='Signal/fmol', ycol='Blocking agent', hue='Salt')

fig = plt.figure()
plt.title('1 nM')
stripbox(diluted_1_nM, 'figs/1_nM_signal_per_fmol_vs_blocking_agent_dots.pdf',
         xcol='Signal/fmol', ycol='Blocking agent', hue='Salt')

fig = plt.figure()
plt.title('1 nM')
stripbox(diluted_1_nM, 'figs/1_nM_signal_retention_vs_blocking_agent_dots.pdf',
         xcol='Retention', ycol='Blocking agent', hue='Salt')



In [ ]:
###
# Signal decay in dotting plotting on nylon or nitrocellulose.
###

df = utils.load_sheet(
    fname='Shift based quantification protocol and results.xlsx',
    sheet_name='180429_2')


nylon = df[df['Membrane']=='Nylon']
nc = df[df['Membrane']=='Nitrocellulose']

def normalize(df):
    aL3_init = df.groupby(['Time (minutes)', 'Object']
                         ).mean().loc[0, 'αL3']['Signal/fmol']
    aL5_init = df.groupby(['Time (minutes)', 'Object']
                         ).mean().loc[0, 'αL5']['Signal/fmol']
    to_k = {
        'αL3': aL3_init,
        'αL5': aL5_init
    }
    df['FU/fmol (normalized)'] = [fu/to_k[obj] for fu, obj in zip(
        df['Signal/fmol'], df['Object'])]
    
    return df

nylon = normalize(nylon)
nc = normalize(nc)

obj_color = utils.obj_color
fig, ax = plt.subplots()

sns.barplot(
    data=nylon, y='FU/fmol (normalized)', x='Time (minutes)', hue='Channel',
    palette=obj_color
    )

utils.draw(fig, 'figs/nylon_vs_nitrocellulose_signal_decay_nylon.pdf')

fig, ax = plt.subplots()
sns.barplot(
    data=nc, y='FU/fmol (normalized)', x='Time (minutes)', hue='Channel',
    palette=obj_color)
utils.draw(fig, 'figs/nylon_vs_nitrocellulose_signal_decay_nitrocellulose.pdf')

In [ ]:
###
# Unused, possibly wrong. Effect of carrier on oligo fluorescence in TBE gels.
###

df = utils.load_sheet(
    fname='Shift based quantification protocol and results.xlsx',
    sheet_name='180425_1')

df['Carrier (ng)'] = df['Carrier']

plt.clf()

fig, ax = plt.subplots()
aL = df.loc[[(x in ['αL5', 'αL3']) for x in df.Object]]
sns.barplot(
    data=aL, y='FU/fmol', x='Carrier (ng)', hue='Object')
fig.savefig('figs/effect_of_carrier_on_lane.pdf')
plt.show()
plt.clf()
plt.close()